In [1]:
import numpy as np
import pandas as pd
import weakref

# Core

In [2]:
# Variable의 역할
## 현재 변수의 이전 노드 함수(한단께 앞 연산 함수)를 저장한다
## 현재 변수의 뒤에서 전달해온 gradient값을 저장한다(gy_list)
## 현재 변수의 gradient값을 연산한다(func.backward())
## 현재 변수의 gradient값을 저장한다(gx_list -> Variable.grad)
## 현재 변수의 세대를 저장한다.(self.generation)

In [3]:
class Variable():
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.node_function = None
        self.generation = 0
        
    def set_node_function(self, func): 
        # Variable을 생성한 노드 함수를 기억한다.
        # 자동 미분시 그래프 관계를 구축하기 위해 사용한다.
        self.node_function = func
        # generation은 하나의 노드에 두개 이상의 엣지가 연결되어 있을 때(즉, a - b / a - c)
        # b와 c의 역전파(grad)를 가장 먼저 처리하고 a로 역전파를 시행하기 위해 필요한 값이다
        # 즉, b와 c는 a보다 큰 generation값을 동일하게 보유하기 때문에
        # generation 값 기준으로 역전파를 처리하게 되면 반드시 a보다는 먼저 처리하는 것이 보장된다.
        self.generation = func.generation + 1
        
    def resetgrad(self):
        self.grad = None
        
    def backward(self):
        # backward 호출 시 Variable을 생성한 노드 함수(즉, 한단계 앞 연산 함수)를 가져온다. 
        # 역전파 최초 시작시엔 None으로 시작하므로, 1.0으로 이루어진 행렬을 생산한다.
        if self.grad is None:
            self.grad = Variable(np.ones_like(self.data))
        
        seen_set = set()
        funcs = []
        
        # add_func는 variable에 저장되어있는 세대 정보(0,1,2,....)를 기준으로 
        # Function을 정렬한다.
        # 이는 노드가 분할되어 하나의 노드에 엣지가 두개 연결될 시(a - b & a - c)
        # b와 c를 먼저!!!! 처리하고 그 이후에 a로 역전파 되도록 보장한다.
        def add_func(func):
            if func not in seen_set:
                funcs.append(func)
                seen_set.add(func)
                funcs.sort(key = lambda x : x.generation)
        # 스택 탐색 알고리즘을 이용하여 가장 뒷 단계 Variable에서 정의된 노드 함수를 가져온다.
        # 즉, 역전파를 시작하기 위한 초기값을 준비한다.
        add_func(self.node_function)
        
        while funcs:
            func = funcs.pop()
             # 현재단계의 뒤에서 전달해온 gradient값을 리스트로 저장한다.
            gy_list = [output().grad for output in func.output_list]
            # 뒷단계 gradient를 노드 함수(즉, 현 연산 함수)에 투입하여 현재단계 grad를 구한다.
            gx_list = func.backward(*gy_list)
            if not isinstance(gx_list, tuple):
                gx_list = (gx_list,)
            
            # 현 단계의 Variable을 불러와 Variable.grad에 Gradient 연산 결과물을 저장한다.
            for x, gx in zip(func.input_list, gx_list):
                # 동일 변수 사용시 gx가 단순히 덧씌워지는 문제를 해결한다.
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                # 현 단계에 연결되어 있는 노드 함수(즉, 앞 단계 연산 함수)를 다음 node_function으로
                # 저장한다.
                if x.node_function is not None :
                    add_func(x.node_function)
            

In [4]:
# Function 껍데기 함수의 역할
## 입력 변수값을 저장한다
## 출력 변수값을 저장한다
## Variable에 현재 함수를 마킹한다
## 다음 세대 Variable에 알려줄 세대를 기록한다.

In [5]:
class Function():
    def __call__(self, *inputs):
        def as_array(x):
            if np.isscalar(x):
                return np.array(x)
            return x
        
        def as_variable(x):
            if isinstance(x, Variable):
                return x
            return Variable(x)
        
        # Variable을 가져온다.
        inputs = [as_variable(x) for x in inputs]
        x_list = [as_array(i.data) for i in inputs]
        # forward 실시, 이 때 x_list는 이미 i.data로 내부 scalar값들이 노출된 상태다.
        y_list = self.forward(*x_list)
        # 출력 결과물이 tuple이 아니면 tuple로 변환해준다.
        if not isinstance(y_list, tuple):
            y_list = (y_list,)
        # 정전파 함수의 연산 결과를 Variable로 신규 생성한다.
        # 이 과정은 그래프 링크를 만드는데 핵심 역할을 담당한다.
        output_list = [Variable(as_array(y)) for y in y_list]
        
        # 모든 output에 대하여 해당 절차를 수행한다.
        self.generation = np.max([i.generation for i in inputs])
        # 정전파를 수행한 함수를 기억한다.
        for output in output_list:
            output.set_node_function(self)
        self.input_list = inputs
        self.output_list = [weakref.ref(x) for x in output_list]
        if len(output_list) > 1:
            return output_list
        else :
            return output_list[0]
    
    # 정전파 껍데기 함수
    def forward(self, x_list):
        raise NotImplementedError()
    
    # 역전파 껍데기 함수
    def backward(self, gy_list):
        raise NotImplementedError()

In [6]:
x = Variable(np.array(2.0))

In [7]:
class Variable():
    def __init__(self, data):
        self.data = data
        self.node_function = None
        self.generation = 0
        self.grad = None
        
    def set_generation(self, generation): 
        # generation은 하나의 노드에 두개 이상의 엣지가 연결되어 있을 때(즉, a - b / a - c)
        # b와 c의 역전파(grad)를 가장 먼저 처리하고 a로 역전파를 시행하기 위해 필요한 값이다
        # 즉, b와 c는 a보다 큰 generation값을 동일하게 보유하기 때문에
        # generation 값 기준으로 역전파를 처리하게 되면 반드시 a보다는 먼저 처리하는 것이 보장된다.
        self.generation = generation + 1
        
    def resetgrad(self):
        self.grad = None

In [8]:
class Function():
    def __call__(self, *inputs):
        def as_array(x):
            if np.isscalar(x):
                return np.array(x)
            return x
        
        def as_variable(x):
            if isinstance(x, Variable):
                return x
            return Variable(x)
        
        # Variable을 가져온다.
        inputs = [as_variable(x) for x in inputs]
        x_list = [as_array(i.data) for i in inputs]
        # forward 실시, 이 때 x_list는 이미 i.data로 내부 scalar값들이 노출된 상태다.
        y_list = self.forward(*x_list)
        # 출력 결과물이 tuple이 아니면 tuple로 변환해준다.
        if not isinstance(y_list, tuple):
            y_list = (y_list,)
        # 정전파 함수의 연산 결과를 Variable로 신규 생성한다.
        # 이 과정은 그래프 링크를 만드는데 핵심 역할을 담당한다.
        output_list = [Variable(as_array(y)) for y in y_list]
        
        # 모든 output에 대하여 해당 절차를 수행한다.
        generation = np.max([i.generation for i in inputs])

        # Generation을 Variable에 기록한다.
        if 'GRADIENT_TAPE' in globals():
            self.making_gradient_tape(output_list, inputs)
        
        if len(output_list) > 1:
            return output_list
        else :
            return output_list[0]
        
    def making_gradient_tape(self, output, inputs):
        for i in output:
            GRADIENT_TAPE[i] = (self, inputs)

    # 정전파 껍데기 함수
    def forward(self, x_list):
        raise NotImplementedError()
    
    # 역전파 껍데기 함수
    def backward(self, gy_list):
        raise NotImplementedError()

In [9]:
class CalcGradient():
    def __call__(self, *inputs):
        def as_array(x):
            if np.isscalar(x):
                return np.array(x)
            return x
        
        def as_variable(x):
            if isinstance(x, Variable):
                return x
            return Variable(x)
        
        def add_func(func):
            if func not in seen_set:
                funcs.append(func)
                seen_set.add(func)
                funcs.sort(key = lambda x : x.generation)
                
        inputs = [as_variable(x) for x in inputs]
        x_list = [as_array(i.data) for i in inputs]
        self.grad = [Variable(np.ones_like(i)) for i in x_list]

        seen_set = set()
        funcs = []

        # add_func는 variable에 저장되어있는 세대 정보(0,1,2,....)를 기준으로 
        # Function을 정렬한다.
        # 이는 노드가 분할되어 하나의 노드에 엣지가 두개 연결될 시(a - b & a - c)
        # b와 c를 먼저!!!! 처리하고 그 이후에 a로 역전파 되도록 보장한다.

        # 스택 탐색 알고리즘을 이용하여 가장 뒷 단계 Variable에서 정의된 노드 함수를 가져온다.
        # 즉, 역전파를 시작하기 위한 초기값을 준비한다.
        for i in inputs:
            add_func(inputs.node_function)
       
        while funcs:
            func = funcs.pop()
             # 현재단계의 뒤에서 전달해온 gradient값을 리스트로 저장한다.
            gy_list = [output().grad for output in func.output_list]
            # 뒷단계 gradient를 노드 함수(즉, 현 연산 함수)에 투입하여 현재단계 grad를 구한다.
            gx_list = func.backward(*gy_list)
            if not isinstance(gx_list, tuple):
                gx_list = (gx_list,)

            # 현 단계의 Variable을 불러와 Variable.grad에 Gradient 연산 결과물을 저장한다.
            for x, gx in zip(func.input_list, gx_list):
                # 동일 변수 사용시 gx가 단순히 덧씌워지는 문제를 해결한다.
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                # 현 단계에 연결되어 있는 노드 함수(즉, 앞 단계 연산 함수)를 다음 node_function으로
                # 저장한다.
                if x.node_function is not None :
                    add_func(x.node_function)


# 함수

## 미분 기본 공식

In [10]:

# 덧셈
class Add(Function):
    # 정전파 : 두 변수를 더한다.
    def forward(self, x_0, x_1):
        self.x_0_shape = x_0.shape
        self.x_1_shape = x_1.shape
        y = x_0 + x_1
        return y
    # 역전파 : 뒷 단계에서 들어온 그레디언트를 양쪽으로 균등하게 흘려보낸다.
    def backward(self, gy):
        gx0, gx1 = gy, gy
        if self.x_0_shape != self.x_1_shape:
            gx0 = sumto(gx0, self.x_0_shape)
            gx1 = sumto(gx1, self.x_1_shape)
        return gx0 , gx1

# 곱셈
class Mul(Function):
    # 정전파 : 두 변수를 곱한다.
    def forward(self, x_0, x_1):
        self.x_0_shape = x_0.shape
        self.x_1_shape = x_1.shape
        y = x_0 * x_1
        return y
    
    # 역전파 : 방향을 스위치해서 뒷 단계의 그레디언트와 입력 변수를 곱해 흘려보낸다. 
    def backward(self, gy):
        x_0 = self.input_list[0]
        x_1 = self.input_list[1]
        gy0, gy1 = gy, gy
        gx_0 = x_0 * gy0 
        gx_1 = x_1 * gy1
        if self.x_0_shape != self.x_1_shape:
            gx_0 = sumto(gx_0, self.x_0_shape)
            gx_1 = sumto(gx_1, self.x_1_shape)
        return gx_1, gx_0

# 음수 변환
class Neg(Function):
    # 정전파 : 음수로 바꾼다.
    def forward(self, x):
        return -x
    
    # 역전파 : 음수로 바꿔 흘려보낸다.
    def backward(self, gy):
        return -gy

# 뺄셈
class Sub(Function):
    # 정전파 : 두 변수를 뺀다.
    def forward(self, x_0, x_1):
        self.x_0_shape = x_0.shape
        self.x_1_shape = x_1.shape
        y = x_0 - x_1
        return y
    # 역전파 : 앞 변수는 그레디언트를, 뒤 변수는 그레디언트 음수를 흘려보낸다.
    def backward(self, gy):
        gx0, gx1 = gy, gy
        if self.x_0_shape != self.x_1_shape:
            gx0 = sumto(gx0, self.x_0_shape)
            gx1 = sumto(gx1, self.x_1_shape)
        return gx0, -gx1

# 나눗셈
class Div(Function):
    # 정전파 : 변수간 나눗셈을 구한다.
    def forward(self, x_0, x_1):
        self.x_0_shape = x_0.shape
        self.x_1_shape = x_1.shape
        y = x_0 / x_1
        return y
    
    # 역전파 : 앞 변수의 경우 1 / a를, 뒤 변수의 경우 (- a / b **2)를 그레디언트와 곱해 흘려보낸다.
    def backward(self, gy):
        x_0, x_1 = self.input_list
        gx0, gx1 = gy, gy
        gx_0 = (1 / x_1) * gx0
        gx_1 = (- x_0 / (x_1) ** 2) * gx1
        if self.x_0_shape != self.x_1_shape:
            gx_0 = sumto(gx_0, self.x_0_shape)
            gx_1 = sumto(gx_1, self.x_1_shape)

        return gx_0 , gx_1 

# 거듭제곱
class Pow(Function):
    # Function 클래스에 거듭제곱 수를 init으로 정의한다.
    def __init__(self, power):
        self.power = power
    
    # 정전파 : 변수에 거듭제곱을 한다.
    def forward(self, x):
        y = x ** self.power
        return y
    
    #역전파 : power * x ^ (power - 1) 에 그레디언트를 곱해 흘려보낸다.
    def backward(self, gy):
        x = self.input_list[0]
        gx = self.power * x ** (self.power - 1) * gy 
        return gx

def add(x_0, x_1):
    return Add()(x_0, x_1)

def mul(x_0, x_1):
    return Mul()(x_0, x_1)    
    
def neg(x):
    return Neg()(x)

def sub(x_0, x_1):
    return Sub()(x_0,x_1)

def rsub(x_0, x_1):
    return Sub()(x_1, x_0)

def div(x_0, x_1):
    return Div()(x_0, x_1)

def rdiv(x_0, x_1):
    return Div()(x_1, x_0)

def power(x, power):
    return Pow(power)(x)

# 연산 기본 메소드들을 덮어씌워
# Variable과 연관된 연산은 기호(+, -, *, /)만 사용해도
# 우리가 정의한 연산을 수행하도록 대치한다.
Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul
Variable.__neg__ = neg
Variable.__sub__ = sub
Variable.__rsub__ = rsub
Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv
Variable.__pow__ = power

In [11]:
class Sum(Function):
    def __init__(self, axis, keepdims):
        # 합을 실시할 축, keepdims 여부를 인자로 받는다.
        # keepdims란 텐서합 이후 원래의 텐서 요소 차원(즉, (n1, n2, ...))를 그대로 유지할것인가
        self.axis = axis
        self.keepdims = keepdims
        
    def forward(self, x):
        # 원본 모양을 기억한다.
        self.x_shape = x.shape
        # axis와 keepdims로 합을 실시한다.
        y = x.sum(axis = self.axis, keepdims = self.keepdims)
        return y
    
    def backward(self, gy):
        # 
        gy = backward_reshape(gy, 
                              x_shape = self.x_shape, 
                              axis = self.axis,
                              keepdims = self.keepdims)
        gx = broadcast_to(gy, x_shape = self.x_shape)
        return gx
    
def flowsum(x,axis,keepdims):
    return Sum(axis,keepdims)(x)

In [12]:
class MatMul(Function):
    def forward(self, x, w):
        y = np.matmul(x, w)
        return y

    def backward(self, gy):
        x = self.input_list[0]
        w = self.input_list[1]
        gw = matmul(transpose(x), gy)
        gx = matmul(gy, transpose(w))
        return gx, gw
    
def matmul(x,w):
    return MatMul()(x,w)

## 텐서 형상 함수

In [13]:
class Reshape(Function):
    def __init__(self, shape):
        # 변환을 원하는 모양을 지정한다.
        self.shape = shape
        
    def forward(self, x):
        # 원본 모양을 저장해준다.
        self.x_shape = x.shape
        # 모양을 reshpae로 바꾼다.
        y = x.reshape(self.shape)
        return y
    
    def backward(self, gy):
        # 역전파시에는 원본 모양을 복원한다.
        gx = reshape(gy, self.x_shape)
        return gx
        
def reshape(x, shape):
    return Reshape(shape)(x)


In [14]:
class Transpose(Function):
    def __init__(self, shape = None):
        self.shape = shape
    
    def forward(self, x):
        # 원본 텐서의 모양을 저장해둔다.
        print(x)
        self.x_shape = x.shape
        # transpose를 실시한다.
        if self.shape is not None:
            axes_len = len(self.shape)
            inv_axes = tuple(np.argsort([ax % axes_len for ax in self.shape]))
            y = np.transpose(x, inv_axes)
        else :
            y = np.transpose(x)
        return y
    
    def backward(self, gy):
        # Variable 변수를 받으므로, np.transpose가 아닌 GoteoFlow의 transpose를 받는다.
        # 저장되어있던 원본 모양을 복원한다.
        gx = transpose(gy, shape = self.x_shape)
        return gx
    
def transpose(x, shape = None):
    return Transpose(shape)(x)
    

In [15]:
def sum_to(x, shape):
    """Sum elements along axes to output an array of a given shape.
    Args:
        x (ndarray): Input array.
        shape:
    Returns:
        ndarray: Output array of the shape.
    """
    ndim = len(shape)
    lead = x.ndim - ndim
    lead_axis = tuple(range(lead))
    
    axis = tuple([i + lead for i, sx in enumerate(shape) if sx == 1])
    y = x.sum(lead_axis + axis, keepdims=True)
    if lead > 0:
        y = y.squeeze(lead_axis)
    return y


In [16]:
class SumTo(Function):
    def __init__(self, shape):
        # 모양을 바꾸면서 덧연산을 실시할 목표 모양을 지정한다.
        self.shape = shape
        
    def forward(self, x):
        # 원본 모양을 기억한다.
        self.x_shape = x.shape
        # sum_to 함수로 모양을 바꾸며 합을 실시한다.
        y = sum_to(x, self.shape)
        return y
    
    def backward(self, gy):
        # 역전파시엔 sum_to로 인해 바뀌었던 모양을 원본 모양으로 복원한다.
        gx = broadcast_to(gy, self.x_shape)
        return gx
    
def sumto(x, shape):
    return SumTo(shape)(x)

In [17]:
class BroadcastTo(Function):
    def __init__(self, shape):
        self.shape = shape
        
    def forward(self, x):
        self.x_shape = x.shape
        y = broadcast_to(x, self.shape)
        return y
    
    def backward(self, gy):
        gx = sumto(x, self.x_shape)
        return gx
    
def broadcast_to(x,shape):
    return BroadcastTo(shape)(x)

## 특수함수

In [18]:
# 특수함수

class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input_list
        gx = np.exp(x) * gy
        return gx 
    
def exp(x):
    return Exp()(x)

In [19]:
class Sin(Function):
    def forward(self, x):
        y = np.sin(x)
        return y
    
    def backward(self, gy):
        x = self.input_list[0]
        gx = cos(x) * gy
        return gx
    
def sin(x):
    return Sin()(x)

In [20]:
class Cos(Function):
    def forward(self, x):
        y = np.cos(x)
        return y
    
    def backward(self, gy):
        x = self.input_list[0]
        return -sin(x) * gy
    
def cos(x):
    return Cos()(x)

In [21]:
a = Variable(np.array([3.0,2.0,3.0]))
b = Variable(np.array([3.0,2.0,3.0]))
c = Variable(np.array([1.0,2.0,3.0]))

In [72]:
a = Variable(np.array([[1,3],[4,5], [6,8]]))
b = Variable(np.array([[10,11],[13,14],[15,16]]))

In [73]:
class GradientTape():
    def __init__(self):
        global GRADIENT_TAPE
        GRADIENT_TAPE = dict()
    def __enter__(self):
        return self
    def __exit__(self, type, value, traceback):
        global GRADIENT_TAPE
        self.gradient_tape = GRADIENT_TAPE
        del GRADIENT_TAPE
    def CalcGradient(self, tapes = None):
        if tapes is None:
            tapes = self.gradient_tape
        tapes = dict(reversed(tapes.items()))
        def as_array(x):
            if np.isscalar(x):
                return np.array(x)
            return x        
        def as_variable(x):
            if isinstance(x, Variable):
                return x
            return Variable(x)        
        for i, tape in enumerate(tapes.items()):                     
            outputs = tape[0]
            if isinstance(outputs, Variable):
                outputs = [outputs]
            if i == 0:
                for j in outputs:
                    j.grad = Variable(np.ones_like(j.data))                      
            inputs = tape[1][1]
            x_list = [as_array(i.data) for i in inputs]                     
            func = tape[1][0]

            gy_list = [output.grad for output in outputs]

            func.input_list = inputs
            gx_list = func.backward(*gy_list)
            if not isinstance(gx_list, tuple):
                gx_list = (gx_list,)

            for x, gx in zip(inputs, gx_list):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

In [74]:
with GradientTape() as tape:
    d = matmul(transpose(a),b)nariflow_tape
    f = d ** 2 + d * 2

inputs (<__main__.Variable object at 0x000001CEC819DA60>,)
x_list [array([[1, 3],
       [4, 5],
       [6, 8]])]
[[1 3]
 [4 5]
 [6 8]]
func <__main__.Transpose object at 0x000001CEC80B0B20>
inputs (<__main__.Variable object at 0x000001CEC819DCD0>, <__main__.Variable object at 0x000001CEC819D190>)
x_list [array([[1, 4, 6],
       [3, 5, 8]]), array([[10, 11],
       [13, 14],
       [15, 16]])]
func <__main__.MatMul object at 0x000001CEC80B0820>
inputs (<__main__.Variable object at 0x000001CEC819D9A0>,)
x_list [array([[152, 163],
       [215, 231]])]
func <__main__.Pow object at 0x000001CEC80B0AF0>
inputs (<__main__.Variable object at 0x000001CEC819D9A0>, 2)
x_list [array([[152, 163],
       [215, 231]]), array(2)]
func <__main__.Mul object at 0x000001CEC819DAF0>
inputs (<__main__.Variable object at 0x000001CEC819D9D0>, <__main__.Variable object at 0x000001CEC8197EE0>)
x_list [array([[23104, 26569],
       [46225, 53361]], dtype=int32), array([[304, 326],
       [430, 462]])]
func <__m

In [75]:
tape.gradient_tape

{<__main__.Variable at 0x1cec819dcd0>: (<__main__.Transpose at 0x1cec80b0b20>,
  [<__main__.Variable at 0x1cec819da60>]),
 <__main__.Variable at 0x1cec819d9a0>: (<__main__.MatMul at 0x1cec80b0820>,
   <__main__.Variable at 0x1cec819d190>]),
 <__main__.Variable at 0x1cec819d9d0>: (<__main__.Pow at 0x1cec80b0af0>,
  [<__main__.Variable at 0x1cec819d9a0>]),
 <__main__.Variable at 0x1cec8197ee0>: (<__main__.Mul at 0x1cec819daf0>,
   <__main__.Variable at 0x1cec8197610>]),
 <__main__.Variable at 0x1cec8197e20>: (<__main__.Add at 0x1cec819dfa0>,
   <__main__.Variable at 0x1cec8197ee0>])}

In [76]:
tape.CalcGradient()

outputs <__main__.Variable object at 0x000001CEC8197E20>
gy_list [<__main__.Variable object at 0x000001CEC8095280>]
outputs <__main__.Variable object at 0x000001CEC8197EE0>
gy_list [<__main__.Variable object at 0x000001CEC8095280>]
inputs (<__main__.Variable object at 0x000001CEC819D9A0>, <__main__.Variable object at 0x000001CEC8095280>)
x_list [array([[152, 163],
       [215, 231]]), array([[1, 1],
       [1, 1]])]
func <__main__.Mul object at 0x000001CEC819DA30>
inputs (<__main__.Variable object at 0x000001CEC8197610>, <__main__.Variable object at 0x000001CEC8095280>)
x_list [array(2), array([[1, 1],
       [1, 1]])]
func <__main__.Mul object at 0x000001CEC819DA30>
inputs (<__main__.Variable object at 0x000001CEC819D490>,)
x_list [array([[152, 163],
       [215, 231]])]
func <__main__.SumTo object at 0x000001CEC80B0EB0>
inputs (<__main__.Variable object at 0x000001CEC8197FD0>,)
x_list [array([[2, 2],
       [2, 2]])]
func <__main__.SumTo object at 0x000001CEC80B0EB0>
outputs <__main_

In [59]:
f.resetgrad()

In [77]:
a.grad.data

array([[ 6794,  9550],
       [ 8732, 12274],
       [10024, 14090]])

In [78]:
b.grad.data

array([[1626, 1744],
       [3438, 3686],
       [5376, 5764]])

array([[1, 1],
       [1, 1],
       [1, 1]])

In [132]:
b.data

array([[10, 11],
       [13, 14],
       [15, 16]])

In [130]:
a.grad.data

array([[ 4866,  5246],
       [ 9964, 10734],
       [15136, 16306]])

In [450]:
class CalcGradient():
    
    def __call__(self, *inputs):
        def as_array(x):
            if np.isscalar(x):
                return np.array(x)
            return x
        
        def as_variable(x):
            if isinstance(x, Variable):
                return x
            return Variable(x)
        
        def add_func(func):
            if func not in seen_set:
                funcs.append(func)
                seen_set.add(func)
                funcs.sort(key = lambda x : x.generation)
                
        inputs = [as_variable(x) for x in inputs]
        x_list = [as_array(i.data) for i in inputs]
        self.grad = [Variable(np.ones_like(i)) for i in x_list]

        seen_set = set()
        funcs = []

        # add_func는 variable에 저장되어있는 세대 정보(0,1,2,....)를 기준으로 
        # Function을 정렬한다.
        # 이는 노드가 분할되어 하나의 노드에 엣지가 두개 연결될 시(a - b & a - c)
        # b와 c를 먼저!!!! 처리하고 그 이후에 a로 역전파 되도록 보장한다.

        # 스택 탐색 알고리즘을 이용하여 가장 뒷 단계 Variable에서 정의된 노드 함수를 가져온다.
        # 즉, 역전파를 시작하기 위한 초기값을 준비한다.
        for i in inputs:
            add_func(inputs.node_function)
       
        while funcs:
            func = funcs.pop()
             # 현재단계의 뒤에서 전달해온 gradient값을 리스트로 저장한다.
            gy_list = [output().grad for output in func.output_list]
            # 뒷단계 gradient를 노드 함수(즉, 현 연산 함수)에 투입하여 현재단계 grad를 구한다.
            gx_list = func.backward(*gy_list)
            if not isinstance(gx_list, tuple):
                gx_list = (gx_list,)

            # 현 단계의 Variable을 불러와 Variable.grad에 Gradient 연산 결과물을 저장한다.
            for x, gx in zip(func.input_list, gx_list):
                # 동일 변수 사용시 gx가 단순히 덧씌워지는 문제를 해결한다.
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                # 현 단계에 연결되어 있는 노드 함수(즉, 앞 단계 연산 함수)를 다음 node_function으로
                # 저장한다.
                if x.node_function is not None :
                    add_func(x.node_function)

In [451]:
tape.CalcGradient()

(<__main__.Variable object at 0x00000263BC244AC0>, <__main__.Variable object at 0x00000263BB50CFD0>)
[array([[[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]]]), array([[[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]]])]
(<__main__.Variable object at 0x00000263BC244AC0>, <__main__.Variable object at 0x00000263BB50CFD0>)
[array([[[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]]]), array([[[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]]])]
(<__main__.Variable object at 0x00000263BC42BF40>, <__main__.Variable object at 0x00000263BC526B20>)
[array([[ 43,  55,  63],
       [ 95, 122, 140],
       [148, 190, 218]]), array([[[2, 2, 2],
        [2, 2, 2],
        [2, 2, 2]]])]
(<__main__.Variable object at 0x00000263BC9CC760>, <__main__.Variable object at 0x00000263BC526B20>)
[array(2), array([[[2, 2, 2],
        [2, 2, 2],
        [2, 2, 2]]])]
(<__main__.Variable object at 0x00000263BC936A60>,)
[array([[[ 86, 110, 126],
        [190, 244, 280],
        [296, 380, 436]]])]
((<__main__.Varia

AttributeError: 'tuple' object has no attribute 'shape'

In [452]:
a = list(tapes.values())[1][0]

In [291]:
def sphere(x,y):
    z = x ** 2 + y ** 2
    return z

In [223]:
def matyas(x, y):
    z = 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y
    return z

In [224]:
def goldstein(x,y):
    z = (1 + (x + y + 1) ** 2 * (19 - 14 * x + 3 * x **2 - 14 * y + 6 * x * y + 3 * y ** 2)) * \
    (30 + (2 * x - 3 * y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    
    return z

In [225]:
def rosenbrock(x_0, x_1):
    z = 100 * (x_1 - x_0 ** 2) ** 2 + (1 - x_0) ** 2
    return z

In [226]:
?np.transpose

In [227]:
x.backward()

AttributeError: 'NoneType' object has no attribute 'generation'

In [228]:
#z = rosenbrock(x_0, x_1)

In [229]:
#z = goldstein(x_0, x_1)

In [230]:
x_0 = Variable(np.array([[1,2,3],[4,5,6]]))
x_1 = Variable(np.array(1.0))

In [297]:
x_0 = reshape(x_0, [6,])

In [302]:
x_0.backward()

In [304]:
x_0.grad.data

array([1, 1, 1, 1, 1, 1])

In [288]:
def functions(x):
    y = x ** 4 - 2 * x ** 2
    return y

In [289]:
z = functions(x_0)

In [290]:
for i in range(0,2):
    if i == 0:
        z.backward()
        print(x_0.grad.data)
    else:
        gx = x_0.grad
        x_0.resetgrad()
        gx.backward()
        print(x_0.grad.data)

24.0
44.0


In [177]:
gx = x_0.grad

In [178]:
x_0.resetgrad()

In [179]:
gx.backward()

In [180]:
x_0.grad.data

array(0.54030231)